In [1]:
# python -m spacy download en_core_web_lg
# Download and installation successful
# You can now load the model via spacy.load('en_core_web_lg')

In [123]:
import spacy
import pandas as pd
import numpy as np
import pandas as pd
from spacy.vectors import Vectors

In [3]:
# bu model 780 mb büyüklüğünde
nlp = spacy.load('en_core_web_lg')

In [4]:
df = pd.read_csv('./japanese_whisky_review.csv')
df.head()

,Unnamed: 0,Bottle_name,Brand,Title,Review_Content
0,1,The Yamazaki Single Malt Whisky - Distiller’s ...,Yamazaki,Overpriced dissapointment,Dull taste. High price. No finish. Over-hyped ...
1,2,The Yamazaki Single Malt Whisky - Distiller’s ...,Yamazaki,Delicious,Delicious! sugared red fruits and sweet with a...
2,3,The Yamazaki Single Malt Whisky - Distiller’s ...,Yamazaki,Good for beginners. i know cos i am a beginner,I am not a whisky expert but i really love the...
3,4,The Yamazaki Single Malt Whisky - Distiller’s ...,Yamazaki,Yamazaki Tutorial,"This is a terrible Yamazaki. Very young, unsh..."
4,5,The Yamazaki Single Malt Whisky - Distiller’s ...,Yamazaki,Very Nice,First time and I like it - fresh but not thin ...


In [5]:
df.shape

(1130, 5)

In [6]:
df.drop(columns=['Unnamed: 0','Bottle_name'],inplace=True)
df.head()

,Brand,Title,Review_Content
0,Yamazaki,Overpriced dissapointment,Dull taste. High price. No finish. Over-hyped ...
1,Yamazaki,Delicious,Delicious! sugared red fruits and sweet with a...
2,Yamazaki,Good for beginners. i know cos i am a beginner,I am not a whisky expert but i really love the...
3,Yamazaki,Yamazaki Tutorial,"This is a terrible Yamazaki. Very young, unsh..."
4,Yamazaki,Very Nice,First time and I like it - fresh but not thin ...


In [197]:
sentence_vector = [nlp(x).vector for x in df[df['Brand']=='Yamazaki']['Review_Content'].values]

In [198]:
sentence_vector = np.stack( sentence_vector, axis=0 )

In [200]:
sentence_vector.shape

(457, 300)

In [67]:
from sklearn.decomposition import TruncatedSVD

In [145]:
svd = TruncatedSVD(n_components=10)
svd_sentences = svd.fit_transform(sentence_vector)

In [146]:
svd_sentences

array([[ 2.8889973e+00,  2.2345310e-01,  6.0537498e-02, ...,
         6.9616008e-01,  4.2496905e-01, -2.4715742e-01],
       [ 2.4281244e+00,  1.0796108e+00, -4.9393526e-01, ...,
        -1.0113779e-01, -6.8672933e-02,  1.1561264e-01],
       [ 3.4394896e+00, -2.1666817e-01,  9.4507840e-03, ...,
        -1.5305886e-02,  3.2230671e-02, -3.0919972e-03],
       ...,
       [ 3.5944409e+00,  4.6319893e-01,  1.0962107e+00, ...,
        -1.3598756e-01, -2.6959601e-01,  9.4364993e-03],
       [ 2.9300835e+00,  4.1738659e-01, -5.3451645e-01, ...,
        -3.3762324e-01, -1.1544983e-02,  1.3604306e-01],
       [ 3.1513660e+00,  1.9204714e+00, -1.1487272e+00, ...,
         4.7945631e-01, -9.1283709e-01, -6.3845873e-01]], dtype=float32)

In [121]:
from sklearn.metrics.pairwise import cosine_similarity
cos_sim=cosine_similarity(svd_sentences,svd_sentences)

In [176]:
pd.DataFrame(cos_sim)[7].sort_values(ascending=False)[:10]

7      1.000000
153    0.999779
115    0.998892
203    0.998343
150    0.998272
255    0.998188
218    0.998163
0      0.997799
206    0.997755
52     0.997485
Name: 7, dtype: float32

In [217]:
print(df[df['Brand']=='Yamazaki']['Review_Content'][7],'\n')
print(df[df['Brand']=='Yamazaki']['Review_Content'][153])

Nice. I like some vanilla and this has the goods.  

Solid drop.


In [186]:
my_sentece_vec = np.stack([nlp("It is really good. I have never taste such a taste in my life. I will buy two more.").vector])

In [201]:
sentence_vector = np.append(sentence_vector,my_sentece_vec,axis=0)

In [202]:
sentence_vector.shape

(458, 300)

In [203]:
svd_sentences = svd.fit_transform(sentence_vector)
cos_sim=cosine_similarity(svd_sentences,svd_sentences)

In [206]:
pd.DataFrame(cos_sim)[457].sort_values(ascending=False)[:10]

457    1.000000
452    0.996916
2      0.996753
148    0.996623
430    0.996566
170    0.996176
135    0.996080
40     0.996025
132    0.995705
441    0.995704
Name: 457, dtype: float32

In [216]:
print("It is really good. I have never taste such a taste in my life. I will buy two more.",'\n')
print(df[df['Brand']=='Yamazaki']['Review_Content'][452])

It is really good. I have never taste such a taste in my life. I will buy two more. 

I was fortunate enough to win the dram lottery. This was a fantastic dram. Sophisticated, flavorful, lasting. I wish I would have won the bottle, but I feel fortunate to have had a chance to try it.
